# Usage

To use UltraNest in a project:

In [ ]:
import ultranest

Next, we need to define our problem.

Lets say we try to fit a powerlaw to a data .

In [ ]:
import numpy as np
data = np.random.power(2.5, size=40)

This problem has 2 parameters: The powerlaw index and the minimum value:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.hist(data)

In [ ]:
param_names = ['index', 'min']

In UltraNest, the parameter space is defined through a transform
from unit values (0...1) to the physical range:

In [ ]:
def my_prior_transform(cube):
    params = cube.copy()

    # transform for index (first parameter):
    # from 0...1 to 0...10    (uniform prior)
    lo = -10
    hi = 10
    params[0] = cube[0] * (hi - lo) + lo

    # transform for min (second parameter):
    # from 0...1 to 0.1 ... 1000  (log-uniform prior)
    lo = 0.1
    hi = 10
    params[1] = 10**(cube[1] * (np.log10(hi) - np.log10(lo)) + np.log10(lo))

    # for more complex priors, you can use the ppf functions 
    # from scipy.stats, such as scipy.stats.norm(mean, std).ppf

    return params


Next, we need to specify the data likelihood given certain parameters:

In [ ]:
import scipy.stats
from numpy import log

def my_likelihood(params):
    
    index, min = params
    ## set up a powerlaw distribution
    rv = scipy.stats.powerlaw(index, loc=min)

    ## compute log-likelihood for each data point and sum
    like = rv.logpdf(data).sum()
    
    # avoid nan and -inf: return a small number instead
    if not np.isfinite(like):
        return -1e100

    return like


We are now ready to explore this parameter space!

In [ ]:
sampler = ultranest.ReactiveNestedSampler(
        param_names,
        loglike=my_likelihood,
        transform=my_prior_transform,
        )

`ReactiveNestedSampler <ultranest.html#ultranest.integrator.ReactiveNestedSampler>`_ 
has several options to specify what logging and file output it should produce.


In [ ]:
result = sampler.run()
sampler.print_results()

In [ ]:
from ultranest.plot import cornerplot
cornerplot(result)


    


    sampler = ultranest.ReactiveNestedSampler(
        param_names, 
        loglike=my_likelihood, 
        transform=my_prior_transform,
        ## additional parameter properties: 
        # identify circular parameters
        wrapped_params=None,
        # add derived calculations
        derived_param_names=[], 
        # where to store the output
        log_dir=None,
        # whether to continue from existing output
        resume=False,
        # make a new folder for each run?
        append_run_num=True,
        run_num=None,
        num_test_samples=2,
        draw_multiple=True,
        num_bootstraps=30,
        show_status=True
        )


Both `ReactiveNestedSampler <ultranest.html#ultranest.integrator.ReactiveNestedSampler>`_ 
and the `run function <ultranest.html#ultranest.integrator.ReactiveNestedSampler.run>`_ 
have several options to specify what logging and file output they should produce,
and how they should explore the parameter space.








In [ ]:
plt.figure()
plt.xlabel('x')
plt.ylabel('Cumulative probability')
plt.hist(data, histtype='step', cumulative=True, density=True, label='data')

from ultranest.plot import PredictionBand

x_grid = np.linspace(0, 1.4, 40)
band = PredictionBand(x_grid)
ppc_band = PredictionBand(x_grid[:-1])

for index, min in result['samples']:
    rv = scipy.stats.powerlaw(index, loc=min)
    band.add(rv.cdf(x_grid))
    
    # generate data
    simulated_data = np.random.power(index, size=len(data))
    hist, _ = np.histogram(simulated_data, bins=x_grid)
    ppc_band.add(hist.cumsum() / hist.sum())

band.line(color='k')
# add 1 sigma quantile
band.shade(color='k', alpha=0.3)

ppc_band.line(color='g', alpha=0.5)
# add 1 sigma quantile
ppc_band.shade(color='g', alpha=0.1)
